* Zero-Pi circuit space
* Distribution of degeneracy and insensitivity
* Fixing Josephson elements : EcJ == EJ := E0
* 2-D parameter space : E0/Ec & E0/El

In [1]:
from DiSuQ.Torch import models
from torch import tensor,stack
from numpy import arange,linspace,meshgrid,array,log,argsort,pi,logspace
from DiSuQ.utils import plotCompare,plotHeatmap,plotBox
from DiSuQ.Torch.optimization import uniformParameters,truncNormalParameters,initializationSequential
from DiSuQ.Torch import optimization
from DiSuQ.Torch.optimization import lossDegeneracyTarget,lossDegeneracyWeighted
from DiSuQ.Torch.components import indE,capE
from torch import set_num_threads
set_num_threads(32)

In [ ]:
L_ = indE(1e-7); L0 = indE(5*1e-9); print('Inductance Bound(GHz):',L_,L0+L_)
C0 = capE(5*1e-16); C_ = capE(1e-14*5) ; print('Capacitance Bound(GHz):',C_,C0+C_)
CJ0 = capE(5*1e-16/4); CJ_ = capE(1e-14/2) ; print('Shunt Bound(GHz):',CJ_,CJ0+CJ_)
J0 = 25. ; J_ = 5. ; print('Junction Bound(GHz):',J_,J0+J_)

In [ ]:
pairs = {'Lx':'Ly','Cx':'Cy','Jx':'Jy','CJx':'CJy'}

In [ ]:
lower,upper = lossScapeBounds(paths)
El = linspace(lower[0],upper[0],n_scape,endpoint=True)
Ec = linspace(lower[1],upper[1],n_scape,endpoint=True)
scape = {'El':El,'Ec':Ec}

In [ ]:
basis = [7,7,7] ; rep = 'Q'
flux_point = ['Lx','Ly']
circuit = models.zeroPi(basis,Ej=10.,Ec=5.,El=10.,EcJ=50.,sparse=False,symmetry=True,
                        _L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0))
static = circuit.circuitState()

In [ ]:
flux_profile = [{'Lx':tensor(.5),'Ly':tensor(0.0)},{'Lx':tensor(.45),'Ly':tensor(0.0)}]

In [ ]:
optimizer = optimization.OrderingOptimization(circuit,representation=rep)

### Degeneracy

In [ ]:
LossD = optimizer.lossScape(lossFunction,flux_profile,scape,static)